# OTP 개념 정리

- `Spring Security`로 개념 역정리

## ✅ OTP 실무 개념 요약

### 1. **OTP란**

> 일정 시간 또는 이벤트에 따라 단 한 번만 유효한 비밀번호를 생성하는 시스템
>
>
> → 보안성 강화 목적, 2차 인증(MFA)에 자주 사용됨
>

---

## ✅ 주요 OTP 방식 3가지

| 구분 | 설명 | 실무 예시 | 관련 알고리즘 |
| --- | --- | --- | --- |
| 🔹 **S/KEY 방식 (HOTP 원형)** | 해시 체인을 미리 생성해 역순으로 사용 | 예전 텍스트 기반 은행 OTP | 반복 HMAC 해시 |
| 🔹 **HOTP (Event-based)** | 이벤트(로그인 시도 횟수 등)에 따라 카운터 증가 | 하드웨어 OTP 토큰 | `HMAC(K, counter)` |
| 🔹 **TOTP (Time-based)** | 현재 시각 기준 (30초 단위로 변함) | 구글 OTP, MS Authenticator | `HMAC(K, timestamp / 30s)` |

---

## ✅ TOTP 예제 (Python)

In [1]:
import pyotp

# 예: 구글 OTP와 동일 구조
secret = pyotp.random_base32()
totp = pyotp.TOTP(secret)

print("Secret Key:", secret)
print("Current OTP:", totp.now())



Secret Key: XUA2F6ZMUT6X3TPZFU2GHDE2ATH3AY7L
Current OTP: 698572


---

## ✅ 실무 포인트

| 질문 | 답변 |
| --- | --- |
| ❓ OTP가 뭐하는 데 쓰이는 건가요? | 비밀번호 재설정, MFA, 보안 로그인에서 인증코드로 사용됨 |
| ❓ "비밀번호 찾기 → 이메일로 OTP" 도 OTP인가요? | 맞지만 일반적으로는 TOTP 기반 앱/토큰을 의미함 (임시 URL도 일종의 OTP) |
| ❓ OTP는 서버-클라이언트 간 동기화가 필요한가요? | TOTP는 시간 동기화 필요 (서버와 클라이언트 시간이 +-30초 내), HOTP는 이벤트 동기화 필요 |
| ❓ 해킹 가능성은? | 평문 저장 없이 공유된 시드 + 시간 기반으로 생성하므로 피싱을 제외하면 안전함 |

---

## ✅ 시험/용어 정리용 요약 (압축)

- **S/KEY**: 해시 체인 기반 1회용 암호 → 서버는 미리 계산된 해시값만 저장
- **HOTP**: HMAC + counter (서버와 동기화 필요, "이벤트 기반")
- **TOTP**: HMAC + timestamp (서버-클라이언트 시간 동기화 필요, "시간 기반")
- `pyotp.TOTP(secret).now()` = 현재 OTP 생성 (실무/시험 둘 다 암기용)

---

## ✅ 한 줄 요약

> OTP는 HMAC(key, counter or timestamp) 기반으로 만들어지는 1회용 비밀번호이며, TOTP가 현재 가장 일반적 (30초 주기)
>

---

## ✅ S/KEY 구조: 해시체인 기반 OTP

> S/KEY = OTP의 시초 (Bellcore 개발)
>
>
> 해시를 반복 적용해 만들어진 **해시 체인**을 거꾸로 사용하는 방식
>

---

### 🔹 구조 그림

```
패스워드: P

H0 = H(H(H(...H(P))))  ← n번 해시
   = H^n(P)

H1 = H(H(...H(P)))     ← n-1번 해시
   = H^(n-1)(P)

...

Hn = P

=> OTP는 Hn, Hn-1, ..., H1 순서대로 사용됨
```

- **서버는 H0만 저장**
- **클라이언트는 P를 알고 있고, H^n(P)를 계산 가능**

---

### 🔹 인증 흐름 (n = 3 예시)

1. 서버: `H0 = H^3(P)` 저장
2. 클라이언트 1차 로그인 → `H3(P) = P` → `H^2(P)` 전송
    - 서버는 `H(H^2(P)) == H^3(P)` 비교 → OK
3. 서버는 `H^2(P)` 저장 → 다음 로그인 땐 `H(P)` 받아서 또 검증

---

### ✅ 실무적 장점

- 서버는 **패스워드 원문 없이** 인증 가능
- OTP가 **한 번 쓰이면 폐기됨** → 재사용 공격에 안전

---

## ✅ 동기화 / 비동기화 구분

| 개념 | 설명 | 예시 | 실무 주의 |
| --- | --- | --- | --- |
| 🔹 **동기화 방식** | 서버-클라이언트 간 시간(또는 이벤트 수) 동기화 필요 | HOTP, TOTP | 시계 오차 / 카운터 오차 주의 |
| 🔹 **비동기화 방식** | 사전 해시 체인으로 구성됨, 동기화 X | S/KEY | 서버는 미리 해시된 값만 비교 |

---

### ✅ S/KEY는 비동기식!

- 사용자는 다음 OTP를 미리 알고 있음 (`H^k(P)`)
- 서버는 그보다 한 단계 더 해시된 값을 저장 (`H^(k-1)(P)`)
- 시간이나 카운터 동기화 필요 없음 → **비동기 방식**

---

## ✅ 비교 정리

| 구분 | S/KEY | HOTP | TOTP |
| --- | --- | --- | --- |
| 동기화 | ❌ 필요 없음 (비동기) | ✅ 카운터 동기화 | ✅ 시간 동기화 |
| 기반 | 해시 체인 | HMAC(key, counter) | HMAC(key, timestamp) |
| 보안 수준 | 높음 (단방향 해시) | 중간 (카운터 desync 위험) | 중간 (시계 오차 민감) |
| 실무 사용 | 거의 없음 | 일부 하드웨어 토큰 | 대부분 앱 기반 OTP (구글 OTP 등) |

---

## ✅ 실무자용 요약 한 줄

> 🔹 S/KEY는 해시체인 기반 OTP로 동기화 필요 없는 비동기식 인증 시스템이다.
>
>
> 🔹 **TOTP**는 시간 기반 동기화가 필요하고, 실무에서 가장 널리 쓰인다.
>

---

## ✅ Python 코드 예시: S/KEY 해시 체인

In [2]:
import hashlib

def hash_function(data: str) -> str:
    return hashlib.sha256(data.encode()).hexdigest()

def generate_hash_chain(seed: str, count: int) -> list:
    chain = [seed]
    for _ in range(count):
        chain.append(hash_function(chain[-1]))
    return chain[::-1]  # 해시체인을 거꾸로 (H^n → H^(n-1) → ... → H)

# 예시
seed = "mypassword"  # 사용자 비밀 키
chain_length = 5     # OTP 5회분 생성

hash_chain = generate_hash_chain(seed, chain_length)

print("🔐 서버에 저장할 초기값 (H^n):", hash_chain[0])
print("\nOTP 사용 순서:")

for i, otp in enumerate(hash_chain):
    print(f"{i+1}번째 OTP → {otp}")



🔐 서버에 저장할 초기값 (H^n): d90339aeaeb55e1581f7bfd95cc33cb4e287b0efa9e3537ffa3b7cf3ade6015f

OTP 사용 순서:
1번째 OTP → d90339aeaeb55e1581f7bfd95cc33cb4e287b0efa9e3537ffa3b7cf3ade6015f
2번째 OTP → d52057c396710f17564e6cf0a51a51696c3c50b5a453d1b94cb0e36fc6f3f66a
3번째 OTP → 6da243debc217c0b51f4230d603ff9b88da7aefd9c6be59ff6a0e1540f0b39f4
4번째 OTP → 1fd0c883fcfec2463c192fd17d2537029ac0c3eaed64ac6812ca258d5303c741
5번째 OTP → 89e01536ac207279409d4de1e5253e01f4a1769e696db0d6062ca9b8f56767c8
6번째 OTP → mypassword


---

## ✅ 실행 결과 예시

```
🔐 서버에 저장할 초기값 (H^n): a81f2...

OTP 사용 순서:
1번째 OTP → a81f2...
2번째 OTP → 0d1b4...
3번째 OTP → e34c9...
4번째 OTP → 6f3d7...
5번째 OTP → 3b21a...  ← 마지막 OTP는 원래 seed와 동일

```

---

## ✅ 인증 흐름 요약

- 서버는 `H^n(seed)`를 저장하고 기다림
- 클라이언트가 `H^(n-1)(seed)`를 제출하면 서버는 한 번 더 해시해서 비교
- 매 인증 후 서버는 새로운 해시값을 저장 (이전보다 한 단계 덜 해시된 값)

---

## ✅ 실무자용 응용 팁

- 해시 함수는 `sha256`, `sha1`, `md5` 중 선택 가능
- `OTP 코드 대신 QR로 전달`하고, 사용자 앱에서 해시 계산하도록 구현 가능
- 서버는 **stateful** 방식 (상태 기억) 필요 — 현재 해시 단계 저장해야 함

## ✅ Spring Security + Google Authenticator OTP 흐름

### 🔐 실무에선 주로 TOTP 기반

---

## ✅ 전체 구조 요약

```
[사용자 브라우저]
    ↕ 로그인 입력
[Spring Security Filter]
    ↕ (인증 후 2단계 필요)
[OTP Controller]
    ↕
[Google Authenticator 앱] ← QR코드로 secret 공유

```

---

## ✅ 세부 흐름

1. **시크릿 키 생성**

    ```java
    GoogleAuthenticator gAuth = new GoogleAuthenticator();
    GoogleAuthenticatorKey key = gAuth.createCredentials();
    String secret = key.getKey();  // Base32 encoded
    ```

2. **QR코드 생성 (URI 형식)**

    ```
    otpauth://totp/{issuer}:{account}?secret={secret}&issuer={issuer}
    ```

    → 프론트에서 QR로 사용자에게 보여줌 → 사용자가 Google Authenticator 앱에 등록

3. **사용자 입력 OTP 수신 후 검증**

    ```java
    boolean isCodeValid = gAuth.authorize(userSecret, otpInputFromUser);

    ```


---

## ✅ 내부적으로는 이렇게 동작해요

```java
// GoogleAuthenticator 내부 구현 방식 (TOTP)
long timeWindow = System.currentTimeMillis() / 30000;
String otp = HMAC(secret, timeWindow);  // HMAC-SHA1/256 기반
```

- 시간은 30초 단위 (TOTP)
- HMAC 기반 OTP → `Truncate → 6자리 숫자`

---

## ✅ Spring이 직접 해주는 건?

| 역할 | 설명 |
| --- | --- |
| 🔸 인증 필터 | 로그인 후 OTP 추가 입력으로 흐름 제어 (ex. 2차 인증 화면 redirect) |
| 🔸 세션 저장 | secret, 시도 횟수 등 세션에 저장하거나 DB 연동 |
| 🔸 예외 처리 | OTP 실패 시 별도 오류 페이지 or 재요청 처리 |
| 🔸 라이브러리 | 대부분 `com.warrenstrange.googleauth.GoogleAuthenticator` 사용 |

---

## ✅ 실무에서 기억할 핵심

- ✅ 직접적으로 `TOTP`를 구현하진 않음 → 라이브러리 래핑
- ✅ `HMAC(K, timestamp/30)` 기반
- ✅ 클라이언트와 서버는 **secret key를 QR로 공유**
- ❗️**Spring Security는 OTP를 완전히 통합 지원하진 않음** → 커스터마이징 필요

---

## ✅ 시험/면접용 요약

| 질문 | 답변 |
| --- | --- |
| Spring에서 OTP는 어떻게 구현하나요? | 보통 Google Authenticator와 TOTP 연동 |
| 직접 구현하나요? | 아니요. `GoogleAuthenticator` 라이브러리를 래핑해서 사용 |
| 내부 동작은? | secret 공유 후, 30초마다 바뀌는 시간 기반 HMAC 코드 생성 |
| 이벤트 동기화 방식도 써봤나요? | 보통은 안 씀. TOTP가 기본이고 HOTP는 하드웨어 토큰에 쓰임 |

---

실제 Spring Security + OTP 예제 프로젝트 구조

→ `로그인 필터`, `OTP 입력 페이지`, `검증 필터`로 이어지는 구조가 흔합니다.

## ✅ 예제 프로젝트 구조: Google Authenticator 기반 OTP 로그인

```
src/
├── config/
│   └── SecurityConfig.java          ← Spring Security 설정
├── controller/
│   ├── LoginController.java         ← 기본 로그인 및 OTP 화면 라우팅
│   └── OTPController.java           ← OTP 입력 및 검증 처리
├── service/
│   └── OTPService.java              ← OTP 생성 및 검증 로직
├── model/
│   └── User.java                    ← 유저 객체 (secret 저장 포함)
├── repository/
│   └── UserRepository.java          ← secret 저장용 (DB or Memory)
└── templates/
    ├── login.html
    └── otp.html
```

---

## ✅ 흐름 요약

1. 사용자가 `login.html`에서 ID/PW 입력
2. 서버는 로그인 성공 후 해당 사용자에게 `otp.html` 페이지로 리다이렉트
3. `otp.html`에 OTP 입력 → 서버가 검증 → 인증 완료 or 실패

---

## ✅ 핵심 코드 구성

### 🔸 1. Secret 키 생성 & QR코드 제공 (`OTPService`)

```java
public String generateSecretKey() {
    GoogleAuthenticator gAuth = new GoogleAuthenticator();
    GoogleAuthenticatorKey key = gAuth.createCredentials();
    return key.getKey();  // Base32 문자열
}

public String getQRBarcodeURL(String user, String secret) {
    return String.format("otpauth://totp/%s?secret=%s&issuer=MyApp", user, secret);
}
```

---

### 🔸 2. OTP 검증 (`OTPService`)

```java
public boolean verifyOTP(String secret, int otp) {
    GoogleAuthenticator gAuth = new GoogleAuthenticator();
    return gAuth.authorize(secret, otp);  // 내부적으로 HMAC(secret, time_step)
}
```

---

### 🔸 3. Security 설정 (`SecurityConfig.java`)

```java
@Override
protected void configure(HttpSecurity http) throws Exception {
    http
        .authorizeRequests()
            .antMatchers("/login", "/otp").permitAll()
            .anyRequest().authenticated()
        .and()
        .formLogin()
            .loginPage("/login")
            .defaultSuccessUrl("/otp", true)  // 로그인 성공 후 OTP 페이지로
        .and()
        .logout().permitAll();
}

```

---

### 🔸 4. OTP Controller 예시

```java
@PostMapping("/otp")
public String processOTP(@RequestParam String username, @RequestParam int otp) {
    String secret = userRepository.getSecret(username);

    if (otpService.verifyOTP(secret, otp)) {
        // 인증 완료 → 세션 등록
        return "redirect:/dashboard";
    } else {
        return "redirect:/otp?error=true";
    }
}
```

---

## ✅ 실무 팁

| 포인트 | 설명 |
| --- | --- |
| Secret 키 저장 | 유저 생성 시 DB에 저장 또는 세션 저장 |
| QR코드 생성 | 클라이언트에 QR Code URL (`otpauth://...`)로 전달 |
| 2단계 인증 흐름 | Spring Security 필터로 구성하거나, controller 기반 redirect |
| 라이브러리 | `com.warrenstrange:googleauth` 사용 권장 |

---

## ✅ Maven 의존성 추가

```xml
<dependency>
    <groupId>com.warrenstrange</groupId>
    <artifactId>googleauth</artifactId>
    <version>1.5.0</version>
</dependency>
```

---

## ✅ 예시 시나리오

1. 회원가입 시 `secret key` 생성 & 저장
2. QR 코드로 Google Authenticator에 등록
3. 로그인 후 → OTP 입력 페이지로 리다이렉트
4. OTP 맞으면 `세션 인증 완료` 후 서비스 진입

---

## ✅ 참고

| 항목 | 내용 |
| --- | --- |
| 공식 URL | `otpauth://totp/{앱명}:{유저}?secret={시크릿}&issuer={앱명}` |
| QR 생성 | `Google Charts API` 또는 프론트 라이브러리 사용 가능 |
| TOTP 동작 | `HMAC(secret, timestamp // 30)` 기반 → `6자리 숫자 추출` |

---